# o'reillyのツノガレイ強化学習の本

## GPIの手法：モンテカルロ法、TD法

## [目次](TableOfContents.ipynb)
- [環境準備](#環境準備)
  - [インストール](#インストール)
  - [インポート](#インポート)
  - [共通関数](#共通関数)
- [モンテカルロ法](#モンテカルロ法)
  - [サンプルモデルで期待値を計算](#サンプルモデルで期待値を計算)
  - [モンテカルロ法の方策反復法](#モンテカルロ法の方策反復法)
    - [状態価値関数を使った方策反復法A](#状態価値関数を使った方策反復法A)
      - [エージェント実装A](#エージェント実装A)
      - [グリッド・ワールド上でエージェントを実行A](#グリッド・ワールド上でエージェントを実行A)
    - [行動価値関数を使った方策反復法B](#行動価値関数を使った方策反復法B)
      - [エージェントの実装B](#エージェントの実装B)
      - [グリッド・ワールド上でエージェントを実行B](#グリッド・ワールド上でエージェントを実行B)
- [方策オフのモンテカルロ法](#方策オフのモンテカルロ法)
  - [重点サンプリング](#重点サンプリング)
  - [方策オフのモンテカルロ法の方策反復法](#方策オフのモンテカルロ法の方策反復法)
    - [状態価値関数を使った方策反復法C](#状態価値関数を使った方策反復法C)
    - [行動価値関数を使った方策反復法D](#行動価値関数を使った方策反復法D)
      - [エージェント実装D](#エージェント実装D)
      - [グリッド・ワールド上でエージェントを実行D](#グリッド・ワールド上でエージェントを実行D)
- [TD法（時間的差分学習法）](#TD法（時間的差分学習法）)
  - [...](#...)
  - [TD法の方策反復法](#TD法の方策反復法)
    - [状態価値関数を使った方策反復法E](#状態価値関数を使った方策反復法E)
      - [エージェント実装E](#エージェント実装E)
      - [グリッド・ワールド上でエージェントを実行E](#グリッド・ワールド上でエージェントを実行E)
    - [行動価値関数を使った方策反復法(Q学習)F](#行動価値関数を使った方策反復法(Q学習)F)
      - [エージェント実装F](#エージェント実装F)
      - [グリッド・ワールド上でエージェントを実行F](#グリッド・ワールド上でエージェントを実行F)
    - [行動価値関数を使った方策反復法(方策オフQ学習)G](#行動価値関数を使った方策反復法(方策オフQ学習)G)
      - [エージェント実装G](#エージェント実装G)
      - [グリッド・ワールド上でエージェントを実行G](#グリッド・ワールド上でエージェントを実行G)
    - [行動価値関数を使った方策反復法(SARSA)H](#行動価値関数を使った方策反復法(SARSA)H)
      - [エージェント実装H](#エージェント実装H)
      - [グリッド・ワールド上でエージェントを実行H](#グリッド・ワールド上でエージェントを実行H)
    - [行動価値関数を使った方策反復法(方策オフのSARSA)I](#行動価値関数を使った方策反復法(方策オフのSARSA)I)
      - [エージェント実装I](#エージェント実装I)
      - [グリッド・ワールド上でエージェントを実行I](#グリッド・ワールド上でエージェントを実行I)
      
## 参考
- https://github.com/oreilly-japan/deep-learning-from-scratch-4/tree/master/ch01
- [強化学習（Reinforcement Learning） - .NET 開発基盤部会 Wiki](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E5%BC%B7%E5%8C%96%E5%AD%A6%E7%BF%92%EF%BC%88Reinforcement%20Learning%EF%BC%89)

## 環境準備

### インストール

In [ ]:
!pip install numpy
!pip install matplotlib
!pip install dezerogym

### インポート

In [ ]:
import numpy as np
from collections import defaultdict
from collections import deque
from dezerogym.gridworld import GridWorld

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定

### 共通関数
greedy法、貪欲法

In [ ]:
def argmax(xs):
    idxes = [i for i, x in enumerate(xs) if x == max(xs)]
    if len(idxes) == 1:
        return idxes[0]
    elif len(idxes) == 0:
        return np.random.choice(len(xs))
    selected = np.random.choice(idxes)
    return selected

In [ ]:
def greedy_probs(Q, state, epsilon=0, action_size=4):
    qs = [Q[(state, action)] for action in range(action_size)]
    max_action = argmax(qs)  # OR np.argmax(qs)
    base_prob = epsilon / action_size
    action_probs = {action: base_prob for action in range(action_size)}  #{0: ε/4, 1: ε/4, 2: ε/4, 3: ε/4}
    action_probs[max_action] += (1 - epsilon)
    return action_probs

## モンテカルロ法

### サンプルモデルで期待値を計算
サイコロを二回振ったときの合計値の期待値
- サンプル回数を増やせば7に収束していく。
- バンディット問題と同じインクリメンタルな実装

In [ ]:
def sample(dices=2):
    x = 0
    for _ in range(dices):
        x += np.random.choice([1, 2, 3, 4, 5, 6])
    return x

trial = 1000
V, n = 0, 0

for _ in range(trial):
    s = sample()
    n += 1
    V += (s - V) / n
    print(V)

### モンテカルロ法の方策反復法
モンテカルロ法の方策反復法では、状態価値関数か行動価値関数のどちらか一方を使用する。

#### 状態価値関数を使った方策反復法A

##### エージェント実装A

In [ ]:
class RandomAgent:
    def __init__(self):
        self.gamma = 0.9
        self.action_size = 4

        random_actions = {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}
        self.pi = defaultdict(lambda: random_actions) # 方策
        self.V = defaultdict(lambda: 0) # 状態価値関数
        self.cnts = defaultdict(lambda: 0) # カウンタ
        self.memory = []

    # 方策から行動を選択
    def get_action(self, state):
        action_probs = self.pi[state]
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        return np.random.choice(actions, p=probs)

    def add(self, state, action, reward):
        data = (state, action, reward)
        self.memory.append(data)

    def reset(self):
        self.memory.clear()

    # 方策反復（エピソード毎）
    def eval(self):
        G = 0
        
        # 逆から計算していく。
        for data in reversed(self.memory):
            state, action, reward = data
            
            # 方策評価
            
            # ココがインクリメンタルな収益の計算
            G = reward + self.gamma * G
            
            # ココは効率の良い状態価値関数の更新の実装
            self.cnts[state] += 1 # カウンタ
            # エピソード毎の状態毎の収益は、履歴の最新の計算結果を利用して計算していく。
            self.V[state] = self.V[state] + (1 / self.cnts[state]) * (G - self.V[state])

##### グリッド・ワールド上でエージェントを実行A

In [ ]:
# グリッドワールドのインスタンスを作成
env = GridWorld()
# エージェントのインスタンスを作成
agent = RandomAgent()

# 1000エピソード
episodes = 1000
for episode in range(episodes):
    
    # 環境と履歴の初期化
    state = env.reset()
    agent.reset()

    # done=tureまでが1エピソード
    while True:
        
        # ActionでStep
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)
        
        #履歴を格納し
        agent.add(state, action, reward)
        
        # done=tureまで、
        if done:
            agent.eval() # 方策反復（エピソード毎）
            break
            
        # 次の状態
        state = next_state

env.render_v(agent.V)

#### 行動価値関数を使った方策反復法B

##### エージェントの実装B
以下の点を修正する。
- 価値関数は、状態価値関数ではなく方策価値関数を使用する。  
（実装はインデックスｓからインデックスｓ,ａを使って集計する様に変更）
  - 状態価値関数：𝑉𝑛(𝑠)＝(G(1)+G(2)+…+G(𝑛))/n =𝑉𝑛-1(𝑠)+1/𝑛{G(𝑛)-𝑉𝑛-1(𝑠)} 
  - 方策価値関数：Q𝑛(𝑠,𝑎)＝(G(1)+G(2)+…+G(𝑛))/n =Q𝑛-1(𝑠,𝑎)+1/𝑛{G(𝑛)-𝑉𝑛-1(𝑠,𝑎)} 
- 行動選択の戦略
  - 貪欲法を採用：最適解を100％の確率で実行する。
  - ε-貪欲法を採用：最適解を１-ε+ε/4、ソレ以外をε/4の確率に更新し実行。
- ε-貪欲法では、非定常バンディット問題の様に学習率を1/𝑛からαに変更。

In [ ]:
class McAgent:
    def __init__(self):
        self.gamma = 0.9
        self.epsilon = 0.1
        self.alpha = 0.1
        self.action_size = 4

        random_actions = {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}
        self.pi = defaultdict(lambda: random_actions) # 方策
        self.Q = defaultdict(lambda: 0) # 行動価値関数
        self.memory = []

    # 方策から行動を選択
    def get_action(self, state):
        action_probs = self.pi[state]
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        return np.random.choice(actions, p=probs)

    # サンプリングして履歴を追加
    def add(self, state, action, reward):
        data = (state, action, reward)
        self.memory.append(data)

    def reset(self):
        self.memory.clear()

    # 方策反復（エピソード毎）
    def update(self):
        G = 0
        
        # 逆から計算していく。
        for data in reversed(self.memory):
            state, action, reward = data
            key = (state, action)
            
            # 方策評価
            
            # ココがインクリメンタルな収益の計算
            G = reward + self.gamma * G
            
            # ココは効率の良い行動価値関数の更新の実装
            # 行動価値関数は、当該状態毎、上・下・左・右のAction毎に価値がある。
            # 確率分布が変化するため指数移動平均を用いる。
            self.Q[key] = self.Q[key] + self.alpha * (G - self.Q[key])
            
            # 方策改善（方策は、状態毎、上・下・左・右の確率）にε-貪欲法（ε-greedy法）を採用。
            self.pi[state] = greedy_probs(self.Q, state, self.epsilon)

##### グリッド・ワールド上でエージェントを実行B

In [ ]:
# グリッドワールドのインスタンスを作成
env = GridWorld()
# エージェントのインスタンスを作成
agent = McAgent()

# 10000エピソード
episodes = 10000
for episode in range(episodes):
    
    # 環境と履歴の初期化
    state = env.reset()
    agent.reset()

    # done=tureまでが1エピソード
    while True:
        
        # ActionでStep
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)

        # 履歴を格納し
        agent.add(state, action, reward)
        
        # done=tureまで、
        if done:
            agent.update() # 方策反復（エピソード毎）
            break
        
        # 次の状態
        state = next_state

env.render_q(agent.Q)

## 方策オフのモンテカルロ法

### 重点サンプリング

In [ ]:
x = np.array([1, 2, 3])
pi = np.array([0.1, 0.1, 0.8])

# =========== Expectation ==================
e = np.sum(x * pi)
print('E_pi[x]', e)

# =========== Monte Carlo ==================
n = 100
samples = []
for _ in range(n):
    s = np.random.choice(x, p=pi)
    samples.append(s)
print('MC: {:.2f} (var: {:.2f})'.format(np.mean(samples), np.var(samples)))

# =========== Importance Sampling ===========
b = np.array([0.2, 0.2, 0.6])  #b = np.array([1/3, 1/3, 1/3])
samples = []
for _ in range(n):
    idx = np.arange(len(b))  # [0, 1, 2]
    i = np.random.choice(idx, p=b)
    s = x[i]
    rho = pi[i] / b[i]
    samples.append(rho * s)
print('IS: {:.2f} (var: {:.2f})'.format(np.mean(samples), np.var(samples)))

### 方策オフのモンテカルロ法の方策反復法
- モチベーション：行動の選択は、探索と適用のバランスを取りたいが、行動価値関数の評価には適用のみを行いたい。
- 別のサイトでは、前半を貪欲法（greedy法）、後半の学習利用をε-貪欲法（ε-greedy法）と言う方法も提案されていた。

#### 状態価値関数を使った方策反復法C
- モンテカルロ法は、状態価値関数・行動価値関数のいずれかを利用なので
- 方策オフのモンテカルロ法の方策反復法には状態価値関数版は無い。

#### 行動価値関数を使った方策反復法D

##### エージェント実装D

In [ ]:
class McOffPolicyAgent:
    def __init__(self):
        self.gamma = 0.9
        self.epsilon = 0.1
        self.alpha = 0.2
        self.action_size = 4

        random_actions = {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}
        self.pi = defaultdict(lambda: random_actions) # 価値推定用の方策
        self.b = defaultdict(lambda: random_actions) # データ収集用の方策
        self.Q = defaultdict(lambda: 0)
        self.memory = []

    # データ収集用の方策から行動を選択
    def get_action(self, state):
        action_probs = self.b[state]
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        return np.random.choice(actions, p=probs)

    # サンプリングして履歴を追加
    def add(self, state, action, reward):
        data = (state, action, reward)
        self.memory.append(data)

    def reset(self):
        self.memory.clear()

    # 方策反復（エピソード毎）
    def update(self):
        G = 0
        rho = 1

        # 逆から計算していく。
        for data in reversed(self.memory):
            state, action, reward = data
            key = (state, action)

            # 方策評価
            # ココがインクリメンタルな収益の計算
            # rhoが追加されているのがポイント
            G = reward + rho * self.gamma * G
            
            # ココは効率の良い行動価値関数の更新の実装
            # 当該状態毎、上・下・左・右のAction毎に価値がある。
            self.Q[key] = self.Q[key] + self.alpha * (G - self.Q[key])
            
            # 重点サンプリングのρ（重み）が計算されている。
            rho *= self.pi[state][action] / self.b[state][action]
            
            # 2つの方策を其々、別々に更新する
            self.pi[state] = greedy_probs(self.Q, state, epsilon=0) # greedy法
            self.b[state] = greedy_probs(self.Q, state, self.epsilon) # ε-greedy法

##### グリッド・ワールド上でエージェントを実行D
ココの実装はほぼ変更なし（McAgent → McOffPolicyAgentの変更ダケ）

In [ ]:
env = GridWorld()
agent = McOffPolicyAgent()

# 10000エピソード
episodes = 10000
for episode in range(episodes):
    
    # 環境と履歴の初期化
    state = env.reset()
    agent.reset()

    # done=tureまでが1エピソード
    while True:
        
        # ActionでStep
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)

        # 履歴を格納し
        agent.add(state, action, reward)
        
        # done=tureまで、
        if done:
            agent.update() # 方策反復（エピソード毎）
            break

        # 次の状態
        state = next_state

env.render_q(agent.Q)

## TD法（時間的差分学習法）

### ...

### TD法の方策反復法

#### 状態価値関数を使った方策反復法E
[状態価値関数を使った方策反復法A](#状態価値関数を使った方策反復法A)と比較すると良い。

##### エージェント実装E

In [ ]:
class TdAgent:
    def __init__(self):
        self.gamma = 0.9
        self.alpha = 0.01
        self.action_size = 4

        random_actions = {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}
        self.pi = defaultdict(lambda: random_actions) # 方策
        self.V = defaultdict(lambda: 0) # 状態価値関数

    # 方策から行動を選択
    def get_action(self, state):
        action_probs = self.pi[state]
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        return np.random.choice(actions, p=probs)

    # 方策反復（ステップ毎）
    def eval(self, state, reward, next_state, done):
        
        # ゴールの状態価値は0（その前に報酬がある）
        # next_Vは、done=trueなら0、done=tureでないならself.V[next_state]
        next_V = 0 if done else self.V[next_state]
        
        # 方策評価
        # ココが、= TD（時間差分学習）法の実装
        #target = reward + self.gamma * next_V
        #self.V[state] += self.alpha * (target - self.V[state])
        self.V[state] = self.V[state] + self.alpha * ((reward + self.gamma * next_V) - self.V[state])

##### グリッド・ワールド上でエージェントを実行E

In [ ]:
# グリッドワールドのインスタンスを作成
env = GridWorld()
# エージェントのインスタンスを作成
agent = TdAgent()

# 1000エピソード
episodes = 1000
for episode in range(episodes):
    
    state = env.reset() # 環境の初期化

    # done=tureまでが1エピソード
    while True:
        
        # ActionでStep
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)

        # 方策反復（ステップ毎）
        agent.eval(state, reward, next_state, done)
        
        # done=tureまで、
        if done:
            break
        
        # 次の状態
        state = next_state

env.render_v(agent.V)

#### 行動価値関数を使った方策反復法(Q学習)F
- [行動価値関数を使った方策反復法B](#行動価値関数を使った方策反復法B)と比較すると良いが、かなり実装が違うので解り難い。
- 以降の順番は、Q、Q（方策オフ）、SARSA、SARSA（方策オフ）に変更。
- 歴史的にQ → SARSAと登場、Qは方策オフが標準、SARSAは方策オンが標準

##### エージェント実装F

In [ ]:
class QLearningAgent:
    def __init__(self):
        self.gamma = 0.9
        self.alpha = 0.8
        self.epsilon = 0.1
        self.action_size = 4
        self.Q = defaultdict(lambda: 0)

    # ε-greedyで行動を選択
    def get_action(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.action_size)
        else:
            qs = [self.Q[state, a] for a in range(self.action_size)]
            return np.argmax(qs)
    
    # 方策改善（ステップ毎）
    def update(self, state, action, reward, next_state, done):
        
        # next_q_maxを計算
        if done:
            next_q_max = 0
        else:
            next_qs = [self.Q[next_state, a] for a in range(self.action_size)]
            next_q_max = max(next_qs)

        # ココがQ学習
        #target = reward + self.gamma * next_q_max
        #self.Q[state, action] += self.alpha * (target - self.Q[state, action])
        self.Q[state, action] = self.Q[state, action] + self.alpha * ((reward + self.gamma * next_q_max) - self.Q[state, action])

##### グリッド・ワールド上でエージェントを実行F

In [ ]:
env = GridWorld()
agent = QLearningAgent()

# 1000エピソード
episodes = 1000
for episode in range(episodes):
    
    state = env.reset() # 環境の初期化

    # done=tureまでが1エピソード
    while True:
        
        # ActionでStep
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)
        
        # 方策反復（ステップ毎）
        agent.update(state, action, reward, next_state, done)
        
        # done=tureまで、
        if done:
            break
        
        # 次の状態
        state = next_state

env.render_q(agent.Q)

#### 行動価値関数を使った方策反復法(方策オフQ学習)G
- Qは方策オフが標準で重点サンプリングを必要としない。
- 理由は行動価値関数の推定を方策に依存しないgreedyで行うので。

##### エージェント実装G

In [ ]:
class QLearningAgent:
    def __init__(self):
        self.gamma = 0.9
        self.alpha = 0.8
        self.epsilon = 0.1
        self.action_size = 4

        random_actions = {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}
        #self.pi = defaultdict(lambda: random_actions) # 価値推定用の方策
        self.b = defaultdict(lambda: random_actions) # データ収集用の方策
        self.Q = defaultdict(lambda: 0)

    # データ収集用の方策から行動を選択
    def get_action(self, state):
        action_probs = self.b[state]
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        return np.random.choice(actions, p=probs)

    # 方策改善（ステップ毎）
    def update(self, state, action, reward, next_state, done):
        
        # next_q_maxを計算（greedyなのでpiが不要に）
        if done:
            next_q_max = 0
        else:
            next_qs = [self.Q[next_state, a] for a in range(self.action_size)]
            next_q_max = max(next_qs)

        # ココがQ学習
        #target = reward + self.gamma * next_q_max
        #self.Q[state, action] += self.alpha * (target - self.Q[state, action])
        self.Q[state, action] = self.Q[state, action] + self.alpha * ((reward + self.gamma * next_q_max) - self.Q[state, action])

        # 2つの方策を其々、別々に更新する
        #self.pi[state] = greedy_probs(self.Q, state, epsilon=0) # 価値推定用の方策はgreedy
        self.b[state] = greedy_probs(self.Q, state, self.epsilon) # データ収集用の方策はε-greedy

##### グリッド・ワールド上でエージェントを実行G

In [ ]:
env = GridWorld()
agent = QLearningAgent()

# 10000エピソード
episodes = 10000
for episode in range(episodes):
    
    state = env.reset() # 環境の初期化

    # done=tureまでが1エピソード
    while True:
        
        # ActionでStep
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)

        # 方策反復（ステップ毎）
        agent.update(state, action, reward, next_state, done)
        
        # done=tureまで、
        if done:
            break
            
        # 次の状態
        state = next_state

env.render_q(agent.Q)

#### 行動価値関数を使った方策反復法(SARSA)H
SARSAは方策オンが標準

##### エージェント実装H

In [ ]:
class SarsaAgent:
    def __init__(self):
        self.gamma = 0.9
        self.alpha = 0.8
        self.epsilon = 0.1
        self.action_size = 4

        random_actions = {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}
        self.pi = defaultdict(lambda: random_actions) # 方策
        self.Q = defaultdict(lambda: 0) # 行動価値関数
        self.memory = deque(maxlen=2)

    # 方策から行動を選択  
    def get_action(self, state):
        action_probs = self.pi[state]
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        return np.random.choice(actions, p=probs)

    def reset(self):
        self.memory.clear()

    # 方策反復（ステップ毎）
    def update(self, state, action, reward, done):
        
        # ステップの履歴を追加
        self.memory.append((state, action, reward, done))
        
        # SARASなので2ステップ必要
        if len(self.memory) < 2:
            return
        state, action, reward, done = self.memory[0]
        next_state, next_action, _, _ = self.memory[1]
        
        # next_q
        next_q = 0 if done else self.Q[next_state, next_action]
        
        # ココがSARAS
        #target = reward + self.gamma * next_q
        #self.Q[state, action] += (target - self.Q[state, action]) * self.alpha
        self.Q[state, action] = self.Q[state, action] + self.alpha * ((reward + self.gamma * next_q) - self.Q[state, action])
        
        self.pi[state] = greedy_probs(self.Q, state, self.epsilon) # 直greedyでも良いのでは？

##### グリッド・ワールド上でエージェントを実行H

In [ ]:
env = GridWorld()
agent = SarsaAgent()

# 10000エピソード
episodes = 10000
for episode in range(episodes):
    
    # 環境と履歴の初期化
    state = env.reset()
    agent.reset()

    # done=tureまでが1エピソード
    while True:
        
        # ActionでStep
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)

        # 方策反復（ステップ毎）
        agent.update(state, action, reward, done)

        # done=tureまで、
        if done:
            agent.update(next_state, None, None, None) # 方策反復（最後）
            break
        
        # 次の状態
        state = next_state

env.render_q(agent.Q)

#### 行動価値関数を使った方策反復法(方策オフのSARSA)I
方策オフのSARSAでは重点サンプリングが必要になる。

##### エージェント実装I

In [ ]:
class SarsaOffPolicyAgent:
    def __init__(self):
        self.gamma = 0.9
        self.alpha = 0.8
        self.epsilon = 0.1
        self.action_size = 4

        random_actions = {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}
        self.pi = defaultdict(lambda: random_actions) # 価値推定用の方策
        self.b = defaultdict(lambda: random_actions) # データ収集用の方策
        self.Q = defaultdict(lambda: 0)
        self.memory = deque(maxlen=2)

    # データ収集用の方策から行動を選択
    def get_action(self, state):
        action_probs = self.b[state]
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        return np.random.choice(actions, p=probs)

    def reset(self):
        self.memory.clear()

    # 方策反復（ステップ毎）
    def update(self, state, action, reward, done):
        
        # ステップの履歴を追加
        self.memory.append((state, action, reward, done))
        
        # SARASなので2ステップ必要
        if len(self.memory) < 2:
            return
        state, action, reward, done = self.memory[0]
        next_state, next_action, _, _ = self.memory[1]

        # next_q, rho
        if done:
            next_q = 0
            rho = 1
        else:
            # next_q
            next_q = self.Q[next_state, next_action]
            # 重点サンプリングのρ（重み）が計算されている。
            rho = self.pi[next_state][next_action] / self.b[next_state][next_action]
        
        # ココがSARAS（方策オフ）
        #target = rho * (reward + self.gamma * next_q)
        #self.Q[state, action] += (target - self.Q[state, action]) * self.alpha
        self.Q[state, action] = self.Q[state, action] + self.alpha * ((rho * (reward + self.gamma * next_q)) - self.Q[state, action])
        
        # 2つの方策を其々、別々に更新する
        self.pi[state] = greedy_probs(self.Q, state, epsilon=0) # 価値推定用の方策はgreedy
        self.b[state] = greedy_probs(self.Q, state, self.epsilon) # データ収集用の方策はε-greedy

##### グリッド・ワールド上でエージェントを実行I

In [ ]:
env = GridWorld()
agent = SarsaOffPolicyAgent()

# 10000エピソード
episodes = 10000
for episode in range(episodes):
    
    # 環境と履歴の初期化
    state = env.reset()
    agent.reset()

    # done=tureまでが1エピソード
    while True:
        
        # ActionでStep
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)

        # 方策反復（ステップ毎）
        agent.update(state, action, reward, done)

        # done=tureまで、
        if done:
            agent.update(next_state, None, None, None) # 方策反復（最後）
            break
            
        # 次の状態
        state = next_state

env.render_q(agent.Q)